# Lecture 13
## 2. Data Structures

### Monday, October 23rd 2017

### From pointers to Iterators

One can simply follow the `next` pointers to the next **POSITION** in a linked list. This suggests an abstraction of the **position** or pointer to an **iterator**, an abstraction which allows us to treat arrays and linked lists with an identical interface. 

The salient points of this abstraction are:

- the notion of a `next` abstracting away the actual gymnastics of where to go next in a storage system
- the notion of a `first` to a `last` that `next` takes us on a journey from and to respectively

- we already implemented the sequence protocol, but 
- now we suggest an additional abstraction that is more fundamental than the notion of a sequence: the **iterable**.

### Iterators and Iterables in python

Just as a sequence is something implementing `__getitem__` and `__len__`, an **Iterable** is something implementing `__iter__`. 

`__len__` is not needed and indeed may not make sense. 

The following example is taken from Fluent Python

In [38]:
import reprlib
class Sentence:
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __getitem__(self, index):
        return self.words[index] 
    
    def __len__(self):
        #completes sequence protocol, but not needed for iterable
        return len(self.words) 
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [39]:
#sequence'
a= Sentence("Mary had a little lamb whose fleece was white as snow.")
len(a), a[3], a

(11, 'little', Sentence('Mary had a l...hite as snow.'))

In [40]:
min(a), max(a)

('Mary', 'whose')

In [41]:
list(a)

['Mary',
 'had',
 'a',
 'little',
 'lamb',
 'whose',
 'fleece',
 'was',
 'white',
 'as',
 'snow.']

To iterate over an object x, python automatically calls `iter(x)`. An **iterable** is something which, when `iter` is called on it, returns an **iterator**.

(1) if `__iter__` is defined, calls that to implement an iterator.

(2) if not  `__getitem__` starting from index 0

(3) otherwise raise TypeError

Any Python sequence is iterable because they implement `__getitem__`. The standard sequences also implement `__iter__`; for future proofing you should too because  (2) might be deprecated in a future version of python.

This:

In [42]:
for i in a:
    print(i)

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


is implemented something like this:

In [43]:
it = iter(a)
while True:
    try:
        nextval = next(it)
        print(nextval)
    except StopIteration:
        del it
        break

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


`it` is an iterator. 

An iterator defines both `__iter__` and a `__next__` (the first one is only required to make sure an *iterator* IS an *iterable*). 

Calling `next` on an iterator will trigger the calling of `__next__`.

In [44]:
it=iter(a)#an iterator defines `__iter__` and can thus be used as an iterable
for i in it:
    print(i)

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


In [45]:
it = iter(a)
next(it), next(it), next(it)

('Mary', 'had', 'a')

So now we can completely abstract away a sequence in favor an iterable (ie we dont need to support indexing anymore). From Fluent:

In [46]:
class SentenceIterator:
    def __init__(self, words): 
        self.words = words 
        self.index = 0
        
    def __next__(self): 
        try:
            word = self.words[self.index] 
        except IndexError:
            raise StopIteration() 
        self.index += 1
        return word 

    def __iter__(self):
        return self
    
class Sentence:#an iterable
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __iter__(self):
        return SentenceIterator(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [47]:
s2 = Sentence("While we could have implemented `__next__` in Sentence itself, making it an iterator, we will run into the problem of exhausting an iterator'.")

In [48]:
len(s2)

TypeError: object of type 'Sentence' has no len()

In [49]:
for i in s2:
    print(i)

While
we
could
have
implemented
`__next__`
in
Sentence
itself,
making
it
an
iterator,
we
will
run
into
the
problem
of
exhausting
an
iterator'.


In [50]:
s2it=iter(s2)
print(next(s2it))
s2it2=iter(s2)
next(s2it),next(s2it2)

While


('we', 'While')

While we could have implemented `__next__` in Sentence itself, making it an iterator, we will run into the problem of "exhausting an iterator". 

The iterator above keeps state in `self.index` and we must be able to start anew by creating a new instance if we want to re-iterate. Thus the `__iter__` in the iterable, simply returns the `SentenceIterator`.

In [51]:
min(s2), max(s2)

('Sentence', 'will')

Note that min and max will work even though we now DO NOT satisfy the sequence protocol, but rather the ITERABLE protocol, as its a pairwise comparison, which can be handled via iteration. The take home message is that in programming with these iterators, these generlization of pointers, we dont need either the length or indexing to work to implement many algorithms: we have abstracted these away.

## A brief addendum on trees

A tree is:

- a hierarchical data structure that has a bunch of items,
- each of which may have a value
- some of which may point to other such items, and some that dont (leaf nodes)
- each item is pointed to by exactly one other item, with the sole exception of the root.

Trees arise everywhere:

- in parsing of code
- evolutionary trees in biology
- language origin trees
- unix file system
- html tags on this page

Just like with lists, one can consider looking at a tree in two ways: a collection of nodes, or a tree with a root and multiple sub-tree's.

Once again, one can represent trees using the recursive data structures we used to represent linked lists (from cs61a):

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/tree.png)



In [52]:
tree = ((1, 2), 3, 4)
tree - (((1, None), (2, None)), (3, None), (4, None))

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

You could also use a tree in which the nodes all themselves have data. This is often used to represent a binary tree.

In [53]:
class Tree:#from cs61a
    
    def __init__(self, data, left=None, right=None):
        self.entry = data
        self.left = left
        self.right = right
    def __repr__(self):
        args = repr(self.entry)
        if self.left or self.right:
            args += ', {0}, {1}'.format(repr(self.left), repr(self.right))
        return 'Tree({0})'.format(args)

Tree(1,Tree(2), Tree(3, Tree(4)))

Tree(1, Tree(2), Tree(3, Tree(4), None))

Once we do iteration in more detail, we'll talk about traversal mechanisms!